# Importação das bibliotecas

In [7]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
import pytz

import yfinance as yf
import MetaTrader5 as mt5

import kaleido
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Organizar datas de extração

In [13]:
hoje = datetime.now()
hoje

datetime.datetime(2024, 1, 22, 21, 43, 12, 341966)

In [14]:
hoje.weekday()

0

In [15]:
segunda = hoje - timedelta(days = hoje.weekday())
segunda

datetime.datetime(2024, 1, 22, 21, 43, 12, 341966)

In [16]:
segunda.date()

datetime.date(2024, 1, 22)

In [17]:
data_inicio = segunda.date().strftime('%Y-%m-%d')
data_final = (hoje.date()+ timedelta(days = 2)).strftime('%Y-%m-%d')
data_final

'2024-01-24'

In [18]:
def capa_post_weekly():

    semana_ano = segunda.date().strftime('%U')
    ano = segunda.date().strftime('%Y')

    print('Trading com Dados apresenta')
    print('Weekly Report')
    print(f'Semana {semana_ano} de {ano}')
    print(f'{segunda.date().strftime("%d-%m")} a {(segunda.date() + timedelta(days = 4)).strftime("%d-%m")}')

In [20]:
capa_post_weekly()

Trading com Dados apresenta
Weekly Report
Semana 03 de 2024
22-01 a 26-01


# Índices Internacionais

Extração e manipulação dos dados

In [21]:
dict_yf_world_indices = {
    'Bovespa (BVSP)': ['^BVSP','Brasil'],
    'S&P 500 (GSPC)': ['^GSPC', 'EUA'],
    'Dow Jones 30 (DJI)': ['^DJI', 'EUA'],
    'NASDAQ Composite (IXIC)': ['^IXIC', 'EUA'],
    'S&P/TSX Composite (GSPTSE)': ['^GSPTSE', 'Canadá'],
    'S&P/BMV IPC (MXX)': ['^MXX', 'México'],
    'S&P Merval (MERV)': ['^MERV', 'Argentina'],
    'Shanghai Composite (SSEC)': ['000001.SS', 'China'],
    'SZSE Component (SZI)': ['399001.SZ', 'China'],
    'Nikkei 225 (N225)': ['^N225', 'Japão'],
    'Hang Seng (HK50)': ['^HSI', 'Hong Kong'],
    'BSE Sensex 30 (BSESN)': ['^BSESN', 'Índia'],
    'EuroNext 100 (N100)': ['^N100', 'Zona Euro'],
    'DAX (DE30)': ['^GDAXI', 'Alemanha'],
    'FTSE 100 (UK100)': ['^FTSE', 'UK'],
    'CAC 40 (FCHI)': ['^FCHI', 'França'],
    'MOEX Russia (IMOEX)': ['IMOEX.ME', 'Rússia'],
    }

In [26]:
df_yf_world_indices = pd.DataFrame.from_dict(dict_yf_world_indices, orient='index', columns=['Ticker_YF', 'Country'])
df_yf_world_indices

,Ticker_YF,Country
Bovespa (BVSP),^BVSP,Brasil
S&P 500 (GSPC),^GSPC,EUA
Dow Jones 30 (DJI),^DJI,EUA
NASDAQ Composite (IXIC),^IXIC,EUA
S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá
S&P/BMV IPC (MXX),^MXX,México
S&P Merval (MERV),^MERV,Argentina
Shanghai Composite (SSEC),000001.SS,China
SZSE Component (SZI),399001.SZ,China
Nikkei 225 (N225),^N225,Japão


In [27]:
df_yf_world_indices = df_yf_world_indices.reset_index()
df_yf_world_indices = df_yf_world_indices.rename({'index':'Índice'}, axis=1)
df_yf_world_indices

,Índice,Ticker_YF,Country
0,Bovespa (BVSP),^BVSP,Brasil
1,S&P 500 (GSPC),^GSPC,EUA
2,Dow Jones 30 (DJI),^DJI,EUA
3,NASDAQ Composite (IXIC),^IXIC,EUA
4,S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá
5,S&P/BMV IPC (MXX),^MXX,México
6,S&P Merval (MERV),^MERV,Argentina
7,Shanghai Composite (SSEC),000001.SS,China
8,SZSE Component (SZI),399001.SZ,China
9,Nikkei 225 (N225),^N225,Japão


In [28]:
ohlcv_indice = pd.DataFrame(yf.download(df_yf_world_indices.Ticker_YF.values[0], start=data_inicio, end=data_final, progress=False, interval="1d"))
ohlcv_indice

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-22 00:00:00-03:00,127636.320312,127842.59375,125875.648438,126601.546875,126601.546875,0


In [29]:
ohlcv_indice = ohlcv_indice.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
ohlcv_indice["Ticker_YF"] = df_yf_world_indices.Ticker_YF.values[0]
ohlcv_indice

,Open,High,Low,Close,Ticker_YF
Date,,,,,
2024-01-28 00:00:00-03:00,127636.320312,127842.59375,125875.648438,126601.546875,^BVSP


In [30]:
ohlcv_indice["Resultado_%"] = (ohlcv_indice.Close/ohlcv_indice.Open - 1) * 100
ohlcv_indice

,Open,High,Low,Close,Ticker_YF,Resultado_%
Date,,,,,,
2024-01-28 00:00:00-03:00,127636.320312,127842.59375,125875.648438,126601.546875,^BVSP,-0.81072


In [33]:
world_ohlcv = pd.DataFrame()

for i in df_yf_world_indices.Ticker_YF:

    try:
        ohlcv_indice = pd.DataFrame(yf.download(i, start=data_inicio, end=data_final, progress=False, interval="1d"))
        ohlcv_indice = ohlcv_indice.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
        ohlcv_indice["Ticker_YF"] = i
        ohlcv_indice["Resultado_%"] = (ohlcv_indice.Close/ohlcv_indice.Open - 1) * 100
        world_ohlcv = pd.concat([world_ohlcv, ohlcv_indice], axis=0)

    except:
        pass


1 Failed download:
- ^MERV: No data found for this date range, symbol may be delisted

1 Failed download:
- ^BSESN: No data found for this date range, symbol may be delisted

1 Failed download:
- IMOEX.ME: No data found for this date range, symbol may be delisted


In [34]:
world_ohlcv

,Open,High,Low,Close,Ticker_YF,Resultado_%
Date,,,,,,
2024-01-28 00:00:00-03:00,127636.320312,127842.593750,125875.648438,126601.546875,^BVSP,-0.810720
2024-01-28 00:00:00-05:00,4853.419922,4868.410156,4844.049805,4850.430176,^GSPC,-0.061601
2024-01-28 00:00:00-05:00,37919.550781,38109.199219,37911.609375,38001.808594,^DJI,0.216927
2024-01-28 00:00:00-05:00,15393.000000,15438.847656,15333.627930,15360.285156,^IXIC,-0.212531
2024-01-28 00:00:00-05:00,20930.929688,20975.279297,20847.140625,20924.300781,^GSPTSE,-0.031670
2024-01-28 00:00:00-06:00,55418.101562,55434.148438,54629.191406,54723.058594,^MXX,-1.254180
2024-01-28 00:00:00+08:00,2826.503418,2828.320312,2735.372070,2756.339844,000001.SS,-2.482345
2024-01-28 00:00:00+08:00,8769.392578,8769.392578,8430.614258,8479.549805,399001.SZ,-3.305164
2024-01-28 00:00:00+09:00,36605.300781,36877.410156,36605.300781,36792.800781,^N225,0.512221


In [35]:
world_indices_result = pd.merge(df_yf_world_indices, world_ohlcv, on="Ticker_YF", how="outer")
world_indices_result

,Índice,Ticker_YF,Country,Open,High,Low,Close,Resultado_%
0,Bovespa (BVSP),^BVSP,Brasil,127636.320312,127842.593750,125875.648438,126601.546875,-0.810720
1,S&P 500 (GSPC),^GSPC,EUA,4853.419922,4868.410156,4844.049805,4850.430176,-0.061601
2,Dow Jones 30 (DJI),^DJI,EUA,37919.550781,38109.199219,37911.609375,38001.808594,0.216927
3,NASDAQ Composite (IXIC),^IXIC,EUA,15393.000000,15438.847656,15333.627930,15360.285156,-0.212531
4,S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá,20930.929688,20975.279297,20847.140625,20924.300781,-0.031670
5,S&P/BMV IPC (MXX),^MXX,México,55418.101562,55434.148438,54629.191406,54723.058594,-1.254180
6,S&P Merval (MERV),^MERV,Argentina,NaN,NaN,NaN,NaN,NaN
7,Shanghai Composite (SSEC),000001.SS,China,2826.503418,2828.320312,2735.372070,2756.339844,-2.482345
8,SZSE Component (SZI),399001.SZ,China,8769.392578,8769.392578,8430.614258,8479.549805,-3.305164
9,Nikkei 225 (N225),^N225,Japão,36605.300781,36877.410156,36605.300781,36792.800781,0.512221


Figura dos Resultados

In [36]:
df_fig = world_indices_result.sort_values(by="Resultado_%", ascending=False)
df_fig["Color"] = np.where(df_fig["Resultado_%"] < 0, "red", "green")
df_fig

,Índice,Ticker_YF,Country,Open,High,Low,Close,Resultado_%,Color
12,EuroNext 100 (N100),^N100,Zona Euro,1380.380005,1392.670044,1380.380005,1388.040039,0.554922,green
9,Nikkei 225 (N225),^N225,Japão,36605.300781,36877.410156,36605.300781,36792.800781,0.512221,green
14,FTSE 100 (UK100),^FTSE,UK,7461.930176,7503.609863,7455.970215,7487.709961,0.345484,green
2,Dow Jones 30 (DJI),^DJI,EUA,37919.550781,38109.199219,37911.609375,38001.808594,0.216927,green
13,DAX (DE30),^GDAXI,Alemanha,16684.099609,16705.490234,16607.720703,16683.359375,-0.004437,red
4,S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá,20930.929688,20975.279297,20847.140625,20924.300781,-0.031670,red
1,S&P 500 (GSPC),^GSPC,EUA,4853.419922,4868.410156,4844.049805,4850.430176,-0.061601,red
3,NASDAQ Composite (IXIC),^IXIC,EUA,15393.000000,15438.847656,15333.627930,15360.285156,-0.212531,red
15,CAC 40 (FCHI),^FCHI,França,7436.609863,7453.209961,7390.680176,7413.250000,-0.314120,red
0,Bovespa (BVSP),^BVSP,Brasil,127636.320312,127842.593750,125875.648438,126601.546875,-0.810720,red


In [40]:
fig_indices_results = go.Figure()

fig_indices_results.add_trace(go.Bar(
                        x=df_fig['Resultado_%'],
                        y=df_fig['Índice'],
                        marker_color=df_fig['Color'],
                        orientation='h',
                        )
                             )

In [41]:
annotations = []
x_pos = []
y_pos = []
x_neg = []
y_neg = []

for i in range(len(df_fig)):

    if df_fig["Resultado_%"].iloc[i] > 0:
        y_pos.append(df_fig["Índice"].iloc[i])
        x_pos.append(np.round(df_fig["Resultado_%"].iloc[i], decimals=2))
    else:
        y_neg.append(df_fig["Índice"].iloc[i])
        x_neg.append(np.round(df_fig["Resultado_%"].iloc[i], decimals=2))

In [42]:
x_pos

[0.55, 0.51, 0.35, 0.22]

In [43]:
x_pos

[0.55, 0.51, 0.35, 0.22]

In [44]:
result_max = df_fig["Resultado_%"].max()
result_min = df_fig["Resultado_%"].min()

for yd, xd in zip(y_pos, x_pos):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

for yd, xd in zip(y_neg, x_neg):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

In [45]:
fig_indices_results.update_xaxes(
        visible=True,
        showticklabels=True,
        range=[(result_min), (result_max)],
    )

fig_indices_results = fig_indices_results.update_layout(
        title_text="<b>Resultado semanal",
        template="simple_white",
        margin=dict(l=20, r=20, t=70, b=20),
        paper_bgcolor="#f7f8fa",
        width=800,
        height=(50 * len(df_fig)),
        annotations=annotations,
        font=dict(family="Arial", size=17, color="black"),
    )

In [46]:
fig_indices_results

In [48]:
max(list(np.abs(x_neg)) + x_pos)

3.31

In [50]:
max(list(np.abs(x_neg)) + x_pos)*1.25

4.1375

In [51]:
fig_indices_results = go.Figure()

fig_indices_results.add_trace(go.Bar(
                        x=df_fig["Resultado_%"],
                        y=df_fig["Índice"],
                        marker_color=df_fig["Color"],
                        orientation="h",
                        )
                    )


annotations = []
x_pos = []
y_pos = []
x_neg = []
y_neg = []

for i in range(len(df_fig)):

    if df_fig["Resultado_%"].iloc[i] > 0:
        y_pos.append(df_fig["Índice"].iloc[i])
        x_pos.append(np.round(df_fig["Resultado_%"].iloc[i], decimals=2))
    else:
        y_neg.append(df_fig["Índice"].iloc[i])
        x_neg.append(np.round(df_fig["Resultado_%"].iloc[i], decimals=2))
        
        
eixo_range = max(list(np.abs(x_neg)) + x_pos)*1.25
dist_barra = 2

result_max = df_fig["Resultado_%"].max() + eixo_range
result_min = df_fig["Resultado_%"].min() - eixo_range

for yd, xd in zip(y_pos, x_pos):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd + dist_barra,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

for yd, xd in zip(y_neg, x_neg):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd - dist_barra,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

fig_indices_results.update_xaxes(
        visible=True,
        showticklabels=True,
        range=[(result_min), (result_max)],
    )

fig_indices_results = fig_indices_results.update_layout(
        title_text="<b>Resultado semanal",
        template="simple_white",
        margin=dict(l=20, r=20, t=70, b=20),
        paper_bgcolor="#f7f8fa",
        width=800,
        height=(50 * len(df_fig)),
        annotations=annotations,
        font=dict(family="Arial", size=17, color="black"),
    )

fig_indices_results
